# Clasificación de Imágenes en Python

## 1. Importar librerías

In [13]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU
from keras.optimizers import Adagrad

## 2. Cargar las imágenes

In [8]:
dirname = os.path.join(os.getcwd(), '..', '..', 'sportimages')
imgpath = dirname + os.sep

images = []
directories = []
dircount = []
prevRoot = ''
cant = 0

print("Leyendo imágenes de", imgpath)

for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant += 1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            images.append(image)
            b = "Leyendo..." + str(cant)
            print(b, end="\r")
            if prevRoot != root:
                print(root, cant)
                prevRoot = root
                directories.append(root)
                dircount.append(cant)
                cant = 0
dircount.append(cant)

dircount = dircount[1:]
dircount[0] = dircount[0] + 1
print("Directorios leídos:", len(directories))
print("Imágenes en cada directorio:", dircount)
print("Suma total de imágenes en subdirectorios:", sum(dircount))

Leyendo imágenes de c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\americano 1
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\basket 9348
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\beisball 8823
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\boxeo 7752
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\ciclismo 7125
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\f1 7533
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\futbol 5053
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\golf 7617
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Ba

## 3. Crear etiquetas y clases

In [9]:
labels = []
indice = 0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice += 1
print("Cantidad de etiquetas creadas:", len(labels))

deportes = []
indice = 0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice, name[len(name)-1])
    deportes.append(name[len(name)-1])
    indice += 1

y = np.array(labels)
X = np.array(images, dtype=np.uint8)

# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print("Total number of outputs:", nClasses)
print("Output classes:", classes)

Cantidad de etiquetas creadas: 77128
0 americano
1 basket
2 beisball
3 boxeo
4 ciclismo
5 f1
6 futbol
7 golf
8 natacion
9 tenis
Total number of outputs: 10
Output classes: [0 1 2 3 4 5 6 7 8 9]


## 4. Creamos sets de entrenamiento y test, validación y preprocesar

In [11]:
# Mezclar todo y crear los grupos de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(f"Train data shape: {X_train.shape}, {y_train.shape}")
print(f"Test data shape: {X_test.shape}, {y_test.shape}")

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.
X_test = X_test / 255.

# Change the labels from categorical to one-hot encoding
y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)

# Display the change for category label using one-hot encoding
print("Original label:", y_train[0])
print("After conversion to one-hot:", y_train_ohe[0])

X_train, X_valid, train_label, valid_label = train_test_split(X_train, y_train_ohe, test_size=0.2, random_state=13)

print(X_train.shape, X_valid.shape, train_label.shape, valid_label.shape)

Train data shape: (61702, 21, 28, 3), (61702,)
Test data shape: (15426, 21, 28, 3), (15426,)
Original label: 6
After conversion to one-hot: [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
(49361, 21, 28, 3) (12341, 21, 28, 3) (49361, 10) (12341, 10)


## 5. Creamos la red

In [14]:
INIT_LR = 1e-3
epochs = 6
batch_size = 64

sport_model = Sequential()
sport_model.add(Conv2D(
    32,
    kernel_size=(3, 3),
    activation='linear',
    padding='same',
    input_shape=(21, 28, 3)
))
sport_model.add(LeakyReLU(alpha=0.1))
sport_model.add(MaxPooling2D((2, 2), padding='same'))
sport_model.add(Dropout(0.5))

sport_model.add(Flatten())
sport_model.add(Dense(32, activation='linear'))
sport_model.add(LeakyReLU(alpha=0.1))
sport_model.add(Dropout(0.5))
sport_model.add(Dense(nClasses, activation='softmax'))

sport_model.summary()

sport_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=Adagrad(learning_rate=INIT_LR, decay=INIT_LR/100),
    metrics=['accuracy']
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 21, 28, 32)        896       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 21, 28, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 11, 14, 32)        0         
                                                                 
 flatten (Flatten)           (None, 4928)              0         
                                                                 
 dense (Dense)               (None, 32)                157728    
                                                        

## 6. Entrenamos la CNN

In [15]:
sport_train_dropout = sport_model.fit(
    X_train,
    train_label,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_valid, valid_label)
)

# Guardamos la red para reutilizarla en el futuro, sin tener que volverla a entrenar
sport_model.save("sports_mnist.h5py")

Epoch 1/6
772/772 [==============================] - 7s 3ms/step - loss: 2.1435 - accuracy: 0.2314 - val_loss: 1.8546 - val_accuracy: 0.5337
Epoch 2/6
772/772 [==============================] - 2s 2ms/step - loss: 1.8054 - accuracy: 0.3873 - val_loss: 1.5022 - val_accuracy: 0.5917
Epoch 3/6
772/772 [==============================] - 2s 3ms/step - loss: 1.5863 - accuracy: 0.4668 - val_loss: 1.2860 - val_accuracy: 0.6121
Epoch 4/6
772/772 [==============================] - 2s 3ms/step - loss: 1.4401 - accuracy: 0.5146 - val_loss: 1.1509 - val_accuracy: 0.6576
Epoch 5/6
772/772 [==============================] - 2s 3ms/step - loss: 1.3414 - accuracy: 0.5461 - val_loss: 1.0609 - val_accuracy: 0.6807
Epoch 6/6
772/772 [==============================] - 2s 2ms/step - loss: 1.2794 - accuracy: 0.5693 - val_loss: 0.9958 - val_accuracy: 0.7183


INFO:tensorflow:Assets written to: sports_mnist.h5py\assets


INFO:tensorflow:Assets written to: sports_mnist.h5py\assets


## 7. Resultados de la clasificación

In [16]:
test_eval = sport_model.evaluate(X_test, y_test_ohe, verbose=1)

print("Test loss:", test_eval[0])
print("Test accuracy:", test_eval[1])

483/483 [==============================] - 1s 1ms/step - loss: 1.0087 - accuracy: 0.7128
Test loss: 1.0086722373962402
Test accuracy: 0.7127577066421509
